# Neighborly Functions App - Cloud Developer With MS Azure - Udacity Project II

## 1. Add configuration

In [1]:
%run variables.sh

## 2. Create Resources

### 2.1. Login Azure CLI

In [2]:
! az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "f958e84a-92b8-439f-a62d-4f45996b6d07",
    "id": "9ed10bd5-8420-4fdf-b87f-621a5ee7dd47",
    "isDefault": true,
    "managedByTenants": [],
    "name": "UdacityDS - 05",
    "state": "Enabled",
    "tenantId": "f958e84a-92b8-439f-a62d-4f45996b6d07",
    "user": {
      "name": "odl_user_231027@udacityhol.onmicrosoft.com",
      "type": "user"
    }
  }
]


### 2.2. Create Resource Group

In [3]:
! az group create \
    --name $resourceGroup \
    --location $location

{
  "id": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2",
  "location": "southeastasia",
  "managedBy": null,
  "name": "azure-dev-udacity-prj2",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


### 2.3. Create Storage Account

In [4]:
! az storage account create \
    --name $storageAccountName \
    --resource-group $resourceGroup \
    --location $location \
    --sku $storageSku

The public access to all blobs or containers in the storage account will be disallowed by default in the future, which means default value for --allow-blob-public-access is still null but will be equivalent to false.
{\ Finished ..
  "accessTier": "Hot",
  "allowBlobPublicAccess": true,
  "allowCrossTenantReplication": null,
  "allowSharedKeyAccess": null,
  "allowedCopyScope": null,
  "azureFilesIdentityBasedAuthentication": null,
  "blobRestoreStatus": null,
  "creationTime": "2023-04-13T19:01:39.217494+00:00",
  "customDomain": null,
  "defaultToOAuthAuthentication": null,
  "dnsEndpointType": null,
  "enableHttpsTrafficOnly": true,
  "enableNfsV3": null,
  "encryption": {
    "encryptionIdentity": null,
    "keySource": "Microsoft.Storage",
    "keyVaultProperties": null,
    "requireInfrastructureEncryption": null,
    "services": {
      "blob": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2023-04-13T19:01:39.608120+00:00"
      },
      "f

### 2.4. Create Function App

In [5]:
! az functionapp create \
    --name $functionName  \
    --storage-account $storageAccountName \
    --consumption-plan-location $location \
    --resource-group $resourceGroup \
    --functions-version $functionVersion \
    --os-type $functionOsType \
    --runtime $functionRuntime

Your Linux function app 'neighborly-function', that uses a consumption plan has been successfully created but is not active until content is published using Azure Portal or the Functions Core Tools.
Application Insights "neighborly-function" was created for this Function App. You can visit https://portal.azure.com/#resource/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/microsoft.insights/components/neighborly-function/overview to view your Application Insights component
{
  "availabilityState": "Normal",
  "clientAffinityEnabled": false,
  "clientCertEnabled": false,
  "clientCertExclusionPaths": null,
  "clientCertMode": "Required",
  "cloningInfo": null,
  "containerSize": 0,
  "customDomainVerificationId": "782FA85143F7169AA64AF0D5CCE3BBCB52BA6F6869BCC490E439AFEF6573CA58",
  "dailyMemoryTimeQuota": 0,
  "defaultHostName": "neighborly-function.azurewebsites.net",
  "enabled": true,
  "enabledHostNames": [
    "neighborly-function.a

### 2.5. Create a CosmosDB manage MongoDB instance

#### 2.5.1. Create CosmosDB

In [6]:
! az cosmosdb create \
    -n $cosmosdbAccountName \
    -g $resourceGroup \
    --kind $cosmosdbKind \
    --server-version $cosmosdbServerSersion \
    --default-consistency-level $cosmosdbDefaultConsistencyLevel \
    --enable-automatic-failover $cosmosdbEnableAutomaticFailover

{
  "analyticalStorageConfiguration": {
    "schemaType": "FullFidelity"
  },
  "apiProperties": {
    "serverVersion": "4.2"
  },
  "backupPolicy": {
    "migrationState": null,
    "periodicModeProperties": {
      "backupIntervalInMinutes": 240,
      "backupRetentionIntervalInHours": 8,
      "backupStorageRedundancy": "Geo"
    },
    "type": "Periodic"
  },
  "capabilities": [
    {
      "name": "EnableMongo"
    }
  ],
  "capacity": null,
  "connectorOffer": null,
  "consistencyPolicy": {
    "defaultConsistencyLevel": "Eventual",
    "maxIntervalInSeconds": 5,
    "maxStalenessPrefix": 100
  },
  "cors": [],
  "createMode": null,
  "databaseAccountOfferType": "Standard",
  "defaultIdentity": "FirstPartyIdentity",
  "disableKeyBasedMetadataWriteAccess": false,
  "disableLocalAuth": false,
  "documentEndpoint": "https://neighborly-cosmosdb.documents.azure.com:443/",
  "enableAnalyticalStorage": false,
  "enableAutomaticFailover": false,
  "enableCassandraConnector": null,
  "ena

#### 2.5.2. Get CosmosDB Connection String

In [8]:
! az cosmosdb list-connection-strings --name $cosmosdbAccountName --resource-group $resourceGroup

This command has been deprecated and will be removed in a future release. Use 'cosmosdb keys list --type connection-strings' instead.
{
  "connectionStrings": [
    {
      "connectionString": "mongodb://neighborly-cosmosdb:PbhlU4NuveT14I1TuEAbQ2QU50UWCSkQwl0b9sxC4KIFJXdddjwE2NN3hJxMNtOMMFLV8BOJ65KPACDb8DAHCg==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@",
      "description": "Primary MongoDB Connection String"
    },
    {
      "connectionString": "mongodb://neighborly-cosmosdb:gEEDXvOoTK07LItwoiRW2j9zI5ol3ZN5Ho8DUYCVphoPGHCKHJykm1VmiYovgMRaADJ2WashrVxdACDbED0mTg==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@",
      "description": "Secondary MongoDB Connection String"
    },
    {
      "connectionString": "mongodb://neighborly-cosmosdb:AsxPeGq682QKFAbj0rxr288PgVbErdrQ1BtOYEU41k

In [9]:
! az cosmosdb keys list \
    --type connection-strings \
    --name $cosmosdbAccountName \
    --resource-group $resourceGroup \
    --query "connectionStrings[0].connectionString" \
    --output tsv

mongodb://neighborly-cosmosdb:PbhlU4NuveT14I1TuEAbQ2QU50UWCSkQwl0b9sxC4KIFJXdddjwE2NN3hJxMNtOMMFLV8BOJ65KPACDb8DAHCg==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@


1. Get the above connection string.
2. Append it to the variables.sh file.
3. Re-run the following script to update variables

In [10]:
%run variables.sh

#### 2.5.3. Create MongoDB Collections

In [11]:
! printf '[{"key": {"keys": ["_id"]}}]' > idxpolicy-congdinh2023.json

1. Create advertisements collection

In [12]:
! az cosmosdb mongodb collection create \
    -a $cosmosdbAccountName \
    -g $resourceGroup \
    -d $cosmosdbDatabaseName \
    -n advertisements \
    --shard _id \
    --throughput 400 \
    --idx @idxpolicy-congdinh2023.json

{\ Finished ..
  "id": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.DocumentDB/databaseAccounts/neighborly-cosmosdb/mongodbDatabases/neighborlydb/collections/advertisements",
  "location": null,
  "name": "advertisements",
  "options": null,
  "resource": {
    "analyticalStorageTtl": null,
    "etag": null,
    "id": "advertisements",
    "indexes": [
      {
        "key": {
          "keys": [
            "_id"
          ]
        },
        "options": null
      }
    ],
    "rid": null,
    "shardKey": {
      "_id": "Hash"
    },
    "ts": null
  },
  "resourceGroup": "azure-dev-udacity-prj2",
  "tags": null,
  "type": "Microsoft.DocumentDB/databaseAccounts/mongodbDatabases/collections"
}


2. Create posts collection

In [13]:
! az cosmosdb mongodb collection create \
    -a $cosmosdbAccountName \
    -g $resourceGroup \
    -d $cosmosdbDatabaseName \
    -n posts \
    --shard _id \
    --throughput 400 \
    --idx @idxpolicy-congdinh2023.json

{\ Finished ..
  "id": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.DocumentDB/databaseAccounts/neighborly-cosmosdb/mongodbDatabases/neighborlydb/collections/posts",
  "location": null,
  "name": "posts",
  "options": null,
  "resource": {
    "analyticalStorageTtl": null,
    "etag": null,
    "id": "posts",
    "indexes": [
      {
        "key": {
          "keys": [
            "_id"
          ]
        },
        "options": null
      }
    ],
    "rid": null,
    "shardKey": {
      "_id": "Hash"
    },
    "ts": null
  },
  "resourceGroup": "azure-dev-udacity-prj2",
  "tags": null,
  "type": "Microsoft.DocumentDB/databaseAccounts/mongodbDatabases/collections"
}


3. If you want to delete a collection, use:

In [ ]:
! az cosmosdb mongodb collection delete \
    -a $cosmosdbAccountName \
    -d $cosmosdbDatabaseName \
    -n posts \
    -g azure-dev-udacity-prj2 \
    --yes

In [ ]:
! az cosmosdb mongodb collection delete \
    -a $cosmosdbAccountName \
    -d $cosmosdbDatabaseName \
    -n advertisements \
    -g azure-dev-udacity-prj2 \
    --yes

4. View the collection

In [14]:
! az cosmosdb mongodb collection list \
    -a $cosmosdbAccountName \
    -g $resourceGroup \
	-d $cosmosdbDatabaseName

[
  {
    "id": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.DocumentDB/databaseAccounts/neighborly-cosmosdb/mongodbDatabases/neighborlydb/collections/posts",
    "location": null,
    "name": "posts",
    "options": null,
    "resource": {
      "analyticalStorageTtl": null,
      "etag": null,
      "id": "posts",
      "indexes": [
        {
          "key": {
            "keys": [
              "_id"
            ]
          },
          "options": null
        }
      ],
      "rid": null,
      "shardKey": {
        "_id": "Hash"
      },
      "ts": null
    },
    "resourceGroup": "azure-dev-udacity-prj2",
    "tags": null,
    "type": "Microsoft.DocumentDB/databaseAccounts/mongodbDatabases/collections"
  },
  {
    "id": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.DocumentDB/databaseAccounts/neighborly-cosmosdb/mongodbDatabases/neighborlydb/collec

#### 2.5.4. Add data to collection

1. Add data to Advertisements Collection - Connection String from 2.5.2

In [ ]:
# Rerun to get new connectionString after inserting
%run variables.sh

In [15]:
! mongoimport \
    --uri=$connectionString \
    --db=$cosmosdbDatabaseName \
    --collection=advertisements \
    --file='./sample_data/sampleAds.json' \
    --jsonArray

zsh:1: parse error near `&'


2. Add data to Posts Collection - Connection String from 2.5.2

In [ ]:
! mongoimport \
    --uri=$connectionString \
    --db=$cosmosdbDatabaseName \
    --collection=posts \
    --file='./sample_data/samplePosts.json' \
    --jsonArray

In [16]:
from pymongo import MongoClient

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# Create a MongoDB client
client = MongoClient($connectionString)

# Access the database and collection
db = client[f"neighborlydb"]

In [ ]:
advertisements = db["advertisements"]

result = advertisements.find({})

# Print out the query result
for doc in result:
    print(doc)


In [ ]:
posts = db["posts"]

result = posts.find({})

# Print out the query result
for doc in result:
    print(doc)


### 2.6. Create App Service for Front-End

1. List Runtime Environment

In [ ]:
! az webapp list-runtimes

2. Create App Service Plan

In [17]:
! az appservice plan create \
    -n $appServicePlanName \
    -g $resourceGroup \
    -l $location \
    --sku $appSku \
    --is-linux

This subscription has reached the limit of 1 Free Linux app service plan(s) it can create in this region. Please choose a different sku or region.


3. Create App Service

In [18]:
! az webapp create \
    -n $appName \
    -g $resourceGroup \
    --plan $appServicePlanName \
    --runtime $appRuntime

The plan 'neighborly-free' doesn't exist.


## 3. Connect Function with CosmosDB

1. Verify the resources

In [19]:
! az functionapp list  -g $resourceGroup

[
  {
    "appServicePlanId": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.Web/serverfarms/SoutheastAsiaLinuxDynamicPlan",
    "availabilityState": "Normal",
    "clientAffinityEnabled": false,
    "clientCertEnabled": false,
    "clientCertExclusionPaths": null,
    "clientCertMode": "Required",
    "cloningInfo": null,
    "containerSize": 0,
    "customDomainVerificationId": "782FA85143F7169AA64AF0D5CCE3BBCB52BA6F6869BCC490E439AFEF6573CA58",
    "dailyMemoryTimeQuota": 0,
    "defaultHostName": "neighborly-function.azurewebsites.net",
    "enabled": true,
    "enabledHostNames": [
      "neighborly-function.azurewebsites.net",
      "neighborly-function.scm.azurewebsites.net"
    ],
    "extendedLocation": null,
    "hostNameSslStates": [
      {
        "hostType": "Standard",
        "ipBasedSslResult": null,
        "ipBasedSslState": "NotConfigured",
        "name": "neighborly-function.azurewebsites.net",
       

2. Connect

In [20]:
! az functionapp config appsettings set \
  -n $functionName \
  -g $resourceGroup \
  --setting MyDbConnection="mongodb://neighborly-cosmosdb:PbhlU4NuveT14I1TuEAbQ2QU50UWCSkQwl0b9sxC4KIFJXdddjwE2NN3hJxMNtOMMFLV8BOJ65KPACDb8DAHCg==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@"

[
  {
    "name": "FUNCTIONS_WORKER_RUNTIME",
    "slotSetting": false,
    "value": "python"
  },
  {
    "name": "FUNCTIONS_EXTENSION_VERSION",
    "slotSetting": false,
    "value": "~3"
  },
  {
    "name": "AzureWebJobsStorage",
    "slotSetting": false,
    "value": "DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName=congdinh2023prj2;AccountKey=FeP/nhJ6M8Y1bQyI9F2C+r5Ef4rl6ZwJiDqhlZ9o2ZfNIeOrDWQ2vPSPtN0I5pd+WbR+VjlR9buQ+AStb131AA=="
  },
  {
    "name": "WEBSITE_CONTENTAZUREFILECONNECTIONSTRING",
    "slotSetting": false,
    "value": "DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName=congdinh2023prj2;AccountKey=FeP/nhJ6M8Y1bQyI9F2C+r5Ef4rl6ZwJiDqhlZ9o2ZfNIeOrDWQ2vPSPtN0I5pd+WbR+VjlR9buQ+AStb131AA=="
  },
  {
    "name": "WEBSITE_CONTENTSHARE",
    "slotSetting": false,
    "value": "neighborly-function95f9637d405f"
  },
  {
    "name": "APPINSIGHTS_INSTRUMENTATIONKEY",
    "slotSetting": false,
    "value": "116d1a9c-f6b9-426e-a3d

## 4. Deploy Functions

1. Test Functions at Local

In [ ]:
! func start

2. Publish functions

In [ ]:
! func azure functionapp publish $functionName

## 5 Dockerize Functions and Deploy to AKS

### 5.1. Create a Dockerfile

In [ ]:
! func init --docker-only --python

### 5.2. Build the image using the Dockerfile

In [ ]:
! docker build -t $imageName:$imageTag .

In [ ]:
! docker run -p 7071:7071 -it $imageName:$imageTag

### 5.3. Create a repository in ACR service

1. Create Azure container registry

In [21]:
! az acr create --resource-group $resourceGroup --name $containerRegistry --sku Basic

{| Finished ..
  "adminUserEnabled": false,
  "anonymousPullEnabled": false,
  "creationDate": "2023-04-13T20:43:27.797058+00:00",
  "dataEndpointEnabled": false,
  "dataEndpointHostNames": [],
  "encryption": {
    "keyVaultProperties": null,
    "status": "disabled"
  },
  "id": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.ContainerRegistry/registries/neighborlycongdinh2023",
  "identity": null,
  "location": "southeastasia",
  "loginServer": "neighborlycongdinh2023.azurecr.io",
  "name": "neighborlycongdinh2023",
  "networkRuleBypassOptions": "AzureServices",
  "networkRuleSet": null,
  "policies": {
    "azureAdAuthenticationAsArmPolicy": {
      "status": "enabled"
    },
    "exportPolicy": {
      "status": "enabled"
    },
    "quarantinePolicy": {
      "status": "disabled"
    },
    "retentionPolicy": {
      "days": 7,
      "lastUpdatedTime": "2023-04-13T20:43:37.241464+00:00",
      "status": "disabled"
   

2. Login Azure container registry

In [22]:
! az acr login --name $containerRegistry

Login Succeeded


3. Show container registry

In [23]:
# Both these commands will give a same result
! az acr show --name $containerRegistry --query loginServer --output table
#! az acr list --resource-group $resourceGroup --query "[].{acrLoginServer:loginServer}" --output table

Result
---------------------------------
neighborlycongdinh2023.azurecr.io


4. List docker images

In [24]:
# List the local images
! docker images

REPOSITORY                                              TAG       IMAGE ID       CREATED              SIZE
neighborly-function                                     latest    b53fe7e1d066   About a minute ago   1.93GB
neighborlycongdinh2023.azurecr.io/neighborly-function   latest    b38b6b906c90   12 hours ago         1.93GB
neighborlycongdinh2023.azurecr.io/neighborly-function   <none>    a41da662c5ad   13 hours ago         1.93GB


5. Push image to Azure container registry

In [ ]:
# Tag the image with the same name as the ACR respository, else, the push will fail.
# SYNTAX 
# docker tag <name:tag> <ACR-respository>.azurecr.io/<name:tag>
! docker tag $imageName:$imageTag $containerRegistry.azurecr.io/$imageName:$imageTag

- Enable Admin User in Access Keys

In [26]:
! az acr update --name $containerRegistry --admin-enabled true

{
  "adminUserEnabled": true,
  "anonymousPullEnabled": false,
  "creationDate": "2023-04-13T20:43:27.797058+00:00",
  "dataEndpointEnabled": false,
  "dataEndpointHostNames": [],
  "encryption": {
    "keyVaultProperties": null,
    "status": "disabled"
  },
  "id": "/subscriptions/9ed10bd5-8420-4fdf-b87f-621a5ee7dd47/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.ContainerRegistry/registries/neighborlycongdinh2023",
  "identity": null,
  "location": "southeastasia",
  "loginServer": "neighborlycongdinh2023.azurecr.io",
  "name": "neighborlycongdinh2023",
  "networkRuleBypassOptions": "AzureServices",
  "networkRuleSet": null,
  "policies": {
    "azureAdAuthenticationAsArmPolicy": {
      "status": "enabled"
    },
    "exportPolicy": {
      "status": "enabled"
    },
    "quarantinePolicy": {
      "status": "disabled"
    },
    "retentionPolicy": {
      "days": 7,
      "lastUpdatedTime": "2023-04-13T20:43:37.241464+00:00",
      "status": "disabled"
    },
    "softD

- Verify Admin User Status

In [27]:
! az acr show --name $containerRegistry --query "adminUserEnabled"

true


In [28]:
# Go to Container Registry >> Settings >> Access Keys and enable the Admin user. 
# Use those credentials to login from your terminal. 
! docker login $containerRegistry.azurecr.io

Username: ^C


In [ ]:
! docker push $containerRegistry.azurecr.io/$imageName:$imageTag

6. View Azure container registry image

In [29]:
# View the newly pushed image in the ACR respository
! az acr repository list --name $containerRegistry --output table

Result
-------------------
neighborly-function


### 5.3. Create an AKS cluster

1. Create an AKS cluster

In [30]:
! az aks create \
    --name $aksCluster \
    --resource-group $resourceGroup \
    --node-count 2 \
    --generate-ssh-keys \
    --attach-acr $containerRegistry \
    --location $location

AAD role propagation done[############################################]  100.0000%{
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "count": 2,
      "creationData": null,
      "currentOrchestratorVersion": "1.24.10",
      "enableAutoScaling": false,
      "enableEncryptionAtHost": false,
      "enableFips": false,
      "enableNodePublicIp": false,
      "enableUltraSsd": false,
      "gpuInstanceProfile": null,
      "hostGroupId": null,
      "kubeletConfig": null,
      "kubeletDiskType": "OS",
      "linuxOsConfig": null,
      "maxCount": null,
      "maxPods": 110,
      "minCount": null,
      "mode": "System",
      "name": "nodepool1",
      "nodeImageVersion": "AKSUbuntu-1804gen2containerd-202303.22.0",
      "nodeLabels": null,
      "nodePublicIpPrefixId": null,
      "nodeTaints": null,
      "orchestratorVersion": "1.24.10",
      "osDiskSizeGb": 128,
      "osDiskType": "Managed",
      "osSku": "Ubu

2. Get credentials for your container service

In [33]:
# Run on terminal
! az aks get-credentials \
    -n $aksCluster \
    -g $resourceGroup

A different object named neighborlycongdinh2023 already exists in your kubeconfig file.
Overwrite? (y/n): ^C


3. Verify the connection to your cluster and view the cluster nodes using

In [34]:
! kubectl get nodes
#Example output:
#NAME                                STATUS   ROLES   AGE     VERSION
#aks-nodepool1-38114521-vmss000000   Ready    agent   3m47s   v1.21.9
#aks-nodepool1-38114521-vmss000001   Ready    agent   3m45s   v1.21.9

NAME                                STATUS   ROLES   AGE     VERSION
aks-nodepool1-95021527-vmss000000   Ready    agent   3m20s   v1.24.10
aks-nodepool1-95021527-vmss000001   Ready    agent   2m57s   v1.24.10


4. Deploy the App to Kubernetes

In [35]:
! func kubernetes install --namespace keda

=namespace/keda created
Installing KEDA v2 in namespace keda
customresourcedefinition.apiextensions.k8s.io/clustertriggerauthentications.keda.sh created
customresourcedefinition.apiextensions.k8s.io/scaledjobs.keda.sh created
customresourcedefinition.apiextensions.k8s.io/scaledobjects.keda.sh created
customresourcedefinition.apiextensions.k8s.io/triggerauthentications.keda.sh created
serviceaccount/keda-operator created
clusterrole.rbac.authorization.k8s.io/keda-external-metrics-reader created
clusterrole.rbac.authorization.k8s.io/keda-operator created
rolebinding.rbac.authorization.k8s.io/keda-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/keda-hpa-controller-external-metrics created
clusterrolebinding.rbac.authorization.k8s.io/keda-operator created
clusterrolebinding.rbac.authorization.k8s.io/keda-system-auth-delegator created
service/keda-metrics-apiserver created
deployment.apps/keda-metrics-apiserver created
deployment.apps/keda-operator created
apiservice.apire

In [39]:
# Run on terminal
! func kubernetes deploy \
    --name $functionName \
    --registry $containerRegistry.azurecr.io

=Error parsing name. The name used for the deployment and other artifacts in kubernetes


In [ ]:
# Run on terminal
! kubectl apply -f ./deploy.yml

In [40]:
! kubectl config get-contexts

CURRENT   NAME                     CLUSTER                  AUTHINFO                                                    NAMESPACE
*         neighborlycongdinh2023   neighborlycongdinh2023   clusterUser_azure-dev-udacity-prj2_neighborlycongdinh2023   


## 6. Delete Resources

In [ ]:
! az group delete -n $resourceGroup --verbose